# RSI(14) пересек 30 вниз - лонг лимитка на лой этой свечки. выход 4-16 свечек.

In [1]:
from simple.binance import getHistMonth
import numpy as np
from talib import RSI
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly_resampler import FigureWidgetResampler

In [2]:
# parameters
frame = '1h'
period = 14
threshold = 30
duration = 5
fee = 0.05
ticker = 'BTCUSDT'

In [3]:
C = getHistMonth('2022-01', '2022-05', ticker, frame, spot=True)[['Open', 'High', 'Low', 'Close', 'Volume']]
C

,Open,High,Low,Close,Volume
DT,,,,,
2022-01-01 00:00:00,46216.93,46731.39,46208.37,46656.13,1503.33095
2022-01-01 01:00:00,46656.14,46949.99,46574.06,46778.14,943.81539
2022-01-01 02:00:00,46778.14,46928.94,46721.96,46811.77,485.16860
2022-01-01 03:00:00,46811.77,46916.63,46760.12,46813.20,562.88971
2022-01-01 04:00:00,46813.21,46887.33,46591.23,46711.05,861.88389
...,...,...,...,...,...
2022-05-31 19:00:00,31682.09,31784.70,31570.53,31625.71,3215.28839
2022-05-31 20:00:00,31625.71,31813.65,31584.29,31813.65,1277.75606
2022-05-31 21:00:00,31813.65,31831.94,31657.26,31720.01,968.45304


In [4]:
# backtest
C['RSI'] = RSI(C.Close, period)
C['Target'] = C.RSI.rolling(window=2).apply(lambda row: 1 if row[0] > threshold and row[1] < threshold else 0).shift()
C['PrevLow'] = C.Low.shift()

Pos = np.zeros(len(C))
for index, row in enumerate(C.itertuples(index=False)):
    if row.Target > 0 and row.Low < row.PrevLow:
        Pos[index:index+duration] = 1

C['Pos'] = Pos
C['DPos'] = C.Pos.diff()
C['Money'] = (-C.DPos * C.Low).cumsum()
C['Fee'] = (C.DPos * C.Open * fee / 100).abs().cumsum()
C['Balance'] = C.Money + C.Pos * C.Open
C

,Open,High,Low,Close,Volume,RSI,Target,PrevLow,Pos,DPos,Money,Fee,Balance
DT,,,,,,,,,,,,,
2022-01-01 00:00:00,46216.93,46731.39,46208.37,46656.13,1503.33095,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
2022-01-01 01:00:00,46656.14,46949.99,46574.06,46778.14,943.81539,NaN,NaN,46208.37,0.0,0.0,0.00,0.000000,0.00
2022-01-01 02:00:00,46778.14,46928.94,46721.96,46811.77,485.16860,NaN,NaN,46574.06,0.0,0.0,0.00,0.000000,0.00
2022-01-01 03:00:00,46811.77,46916.63,46760.12,46813.20,562.88971,NaN,NaN,46721.96,0.0,0.0,0.00,0.000000,0.00
2022-01-01 04:00:00,46813.21,46887.33,46591.23,46711.05,861.88389,NaN,NaN,46760.12,0.0,0.0,0.00,0.000000,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-05-31 19:00:00,31682.09,31784.70,31570.53,31625.71,3215.28839,53.756059,0.0,31592.33,0.0,0.0,-5389.78,1280.284555,-5389.78
2022-05-31 20:00:00,31625.71,31813.65,31584.29,31813.65,1277.75606,57.736612,0.0,31570.53,0.0,0.0,-5389.78,1280.284555,-5389.78
2022-05-31 21:00:00,31813.65,31831.94,31657.26,31720.01,968.45304,55.187684,0.0,31584.29,0.0,0.0,-5389.78,1280.284555,-5389.78


In [5]:
Buy = C[C.DPos > 0]
Sell = C[C.DPos < 0]

In [6]:
fig = FigureWidgetResampler(go.FigureWidget(make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing=0,
                                    row_heights=[0.7, 0.15, 0.15], specs=[[{"secondary_y": True}], [{}], [{}]])))

fig.update_layout(autosize=True, height=800, template='plotly_white',
                  legend=dict(x=0.1, y=1.01, orientation="h"),
                  margin=dict(l=45, r=5, b=10, t=10, pad=3),
                  xaxis_rangeslider_visible=False,
                  yaxis2_showgrid=False)

fig.add_candlestick(x=C.index, open=C.Open, high=C.High, low=C.Low, close=C.Close, name='OHLC', opacity=0.5)
fig.add_scattergl(name='Buy', mode='markers', x=Buy.index, y=Buy.PrevLow, marker=dict(symbol='triangle-up', color='green', size=10)) 
fig.add_scattergl(name='Sell', mode='markers', x=Sell.index, y=Sell.Open, marker=dict(symbol='triangle-down', color='red', size=10))

fig.add_trace(go.Scattergl(x=C.index, y=C.Balance, name='Balance', line_color='blue', connectgaps=True, line_shape='hv'), secondary_y=True)
fig.add_trace(go.Scattergl(x=C.index, y=C.Fee, name='Fee', line_color='red', connectgaps=True), secondary_y=True)

fig.add_scattergl(name='RSI', x=C.index, y=C.RSI, row=2, col=1)
fig.add_bar(name='Long', x=C.index, y=C.Pos, marker_color='green', row=3, col=1, opacity=0.5)
fig.add_scattergl(x=C.index, y=C.Target/2, name='Target', row=3, col=1, line_color='green', connectgaps=True)

fig.update_xaxes(spikemode='across+marker', spikedash='dot', spikethickness=1, spikesnap='cursor')
fig.update_traces(xaxis='x1')
fig.update_xaxes(showgrid=True, title='Time', visible=True, showticklabels=True);

In [7]:
fig

FigureWidgetResampler({
    'data': [{'close': array([46656.13, 46778.14, 46811.77, ..., 31720.01, 31869.28, 3…